In [1]:
import numpy as np
from scipy.io import netcdf_file
from scipy.interpolate import RegularGridInterpolator as rgi
import scipy.fftpack as fp

In [2]:
fh = netcdf_file('bhesse.nc', 'r', mmap=False)
x1 = fh.variables['x'][:].astype(np.float32)
y1 = fh.variables['y'][:].astype(np.float32)
z1 = fh.variables['z'][:].astype(np.float32)
bx1 = np.swapaxes(fh.variables['bx'][:],0,2).astype(np.float32)
by1 = np.swapaxes(fh.variables['by'][:],0,2).astype(np.float32)
bz1 = np.swapaxes(fh.variables['bz'][:],0,2).astype(np.float32)
fh.close()

dx = np.mean(x1[1:] - x1[:-1])
dy = np.mean(y1[1:] - y1[:-1])
dz = np.mean(z1[1:] - z1[:-1])
nx = np.size(x1, 0)
ny = np.size(y1, 0)
nz = np.size(z1, 0)

bxs = rgi((x1, y1, z1), bx1, bounds_error=False, fill_value=0)
bys = rgi((x1, y1, z1), by1, bounds_error=False, fill_value=0)
bzs = rgi((x1, y1, z1), bz1, bounds_error=False, fill_value=0)

In [3]:
x1.shape

(64,)

In [4]:
bx1.shape

(64, 64, 64)

In [5]:
# Generate cell-centre coordinate arrays, including ghost cells
xc = np.linspace(x1[0]-0.5*dx, x1[-1]+0.5*dx, nx+1)
yc = np.linspace(y1[0]-0.5*dy, y1[-1]+0.5*dy, ny+1)
zc = np.linspace(z1[0]-0.5*dz, z1[-1]+0.5*dz, nz+1)

In [6]:
xc.shape

(65,)

In [7]:
x2, y2 = np.meshgrid(xc[1:-1], yc[1:-1], indexing='ij')
x2.shape

(63, 63)

In [8]:
bz = bzs(np.stack((x2, y2, x2*0 + z1[0]), axis=len(np.shape(x2))))
bz.shape

(63, 63)

In [9]:
b0 = np.mean(bz)
b0

np.float64(4.5413062643046355e-08)

In [10]:
u = fp.fft2(bz)
m, n = np.mgrid[0:nx-1, 0:ny-1]
m, n = np.cos(2*m*np.pi/(nx-1)), np.cos(2*n*np.pi/(ny-1))
u /= 2.0*((m - 1.0)/(dx)**2 + (n - 1.0)/(dy)**2) + 1e-16
u[0,0] = 0
u = np.real(fp.ifft2(u))
u.shape

(63, 63)

In [11]:
ub = np.zeros((nx+1,ny+1))
ub[1:-1,1:-1] = u
ub[0,:] = ub[-2,:]
ub[-1,:] = ub[1,:]
ub[:,0] = ub[:,-2]
ub[:,-1] = ub[:,1]
ub.shape

(65, 65)

In [12]:
a0x = -(ub[1:-1,1:] - ub[1:-1,:-1])/dy
a0y = (ub[1:,1:-1] - ub[:-1,1:-1])/dx
_, y2 = np.meshgrid(xc[1:-1], y1, indexing='ij')
a0x -= 0.5*y2*b0
x2, _ = np.meshgrid(x1, yc[1:-1], indexing='ij')
a0y += 0.5*x2*b0

In [13]:
ax = np.zeros((nx+1, ny, nz))
print(ax.shape)
x3, y3, z3 = np.meshgrid(xc[1:-1], y1, zc[1:-1], indexing='ij')
by = bys(np.stack((x3, y3, z3), axis=len(np.shape(x3))))
print(by.shape)
ax[1:-1,:,:] = np.repeat(np.expand_dims(a0x, axis=-1), nz, axis=2)
ax[1:-1,:,1:] += np.cumsum(by, axis=2)*dz
ax[0,:,:] = ax[1,:,:]
ax[-1,:,:] = ax[-2,:,:]
axs = rgi((xc, y1, z1), ax, bounds_error=False, fill_value=0)
del(ax)

(65, 64, 64)
(63, 64, 63)


In [16]:
x3, y3, z3 = np.meshgrid(xc[1:-1], y1, z1, indexing='ij')
print(x3.shape)

x3, y3, z3 = np.meshgrid(x1, yc[1:-1], z1, indexing='ij')
print(x3.shape)

(63, 64, 64)
(64, 63, 64)
